# Use LangChain with Meta-Llama-3.1-405B-Instruct for Azure AI

You can use Meta-Llama-3.1-405B-Instruct - or any of the Llama 3.1 models - as a serverless API endpoint through Azure AI Studio and Azure ML with `langchain` to create more sophisticated intelligent applications. Models available on Azure as a serverless API endpoint:

- `Meta-Llama-3.1-405B-Instruct`
- `Meta-Llama-3.1-70B-Instruct`
- `Meta-Llama-3.1-8B-Instruct`

The above models are supported by cross-regional support for any region in the US.

> Review the [documentation](https://aka.ms/meta-llama-3.1-405B-instruct-azure-ai-studio-docs) for Meta-Llama-3.1-405B-Instruct and other Llama 3.1 models through AI Studio and for ML Studio for details on how to provision inference endpoints, regional availability, pricing and inference schema reference.

## Prerequisites

Before we start, there are certain steps we need to take to deploy the models:

* Register for a valid Azure account with subscription 
* Make sure you have access to [Azure AI Studio](https://learn.microsoft.com/en-us/azure/ai-studio/what-is-ai-studio?tabs=home)
* Create a project and resource group
* Select `Meta-Llama-3.1-405B-Instruct`.

    > Notice that some models may not be available in all the regions in Azure AI and Azure Machine Learning. On those cases, you can create a workspace or project in the region where the models are available and then consume it with a connection from a different one. To learn more about using connections see [Consume models with connections](https://learn.microsoft.com/en-us/azure/ai-studio/how-to/deployments-connections)

* Deploy with "Pay-as-you-go"

Once deployed successfully, you should be assigned for an API endpoint and a security key for inference.

For more information, you should consult Azure's official documentation [here](https://aka.ms/meta-llama-3.1-405B-instruct-azure-ai-studio-docs) for model deployment and inference.

To complete this tutorial, you will need to:

* Install `langchain` and `langchain_community`

    ```bash
    pip install langchain langchain_community
    ```

## Example

The following example demonstrate how to create a chain that uses `Meta-Llama-3.1-405B-Instruct` model deployed through Azure AI and Azure ML. The chain has been configured with a `ConversationBufferMemory`. This example has been adapted from [LangChain official documentation](https://python.langchain.com/docs/modules/memory/adding_memory).

In [ ]:
%pip install langchain langchain_community

In [ ]:
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain.schema import SystemMessage
from langchain_community.chat_models.azureml_endpoint import (
    AzureMLChatOnlineEndpoint,
    AzureMLEndpointApiType,
    LlamaChatContentFormatter,
)

Let's create an instance of our `AzureMLChatOnlineEndpoint` model. This class allow us to get access to any model deployed in Azure AI or Azure ML. For completion models use class `langchain_community.llms.azureml_endpoint.AzureMLOnlineEndpoint` with `LlamaContentFormatter` as the `content_formatter`.

- `endpoint`: Use the endpoint URL from your deployment. Include `/v1` at the end of the endpoint URL.
- `api_key`: Use your API key.

In [ ]:
chat_model = AzureMLChatOnlineEndpoint(
    endpoint_url="https://<endpoint-name>.<region>.inference.ai.azure.com/v1/chat/completions",
    endpoint_api_type=AzureMLEndpointApiType.serverless,
    endpoint_api_key="<key>",
    content_formatter=LlamaChatContentFormatter(),
)

In the below prompt, we have two input keys: one for the actual input (`human_input`), and another for the input from the `Memory` class (`chat_history`).

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content="You are a chatbot having a conversation with a human. You love making references to pop culture in  your answers."
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{human_input}"),
    ]
)

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

We create the chain as follows:

In [ ]:
chat_llm_chain = LLMChain(
    llm=chat_model,
    prompt=prompt,
    memory=memory,
    verbose=True,
)

We can see how it works:

In [ ]:
chat_llm_chain.predict(human_input="Hi there my friend")

In [ ]:
chat_llm_chain.predict(
    human_input="Who is the most renowned architect in the world? Provide a short poem that describes their work in the style of Shakespeare with Iambic pentimeter and a rhythm pattern of ABABCC"
)

## Aditional resources

Here are some additional reference:  

* [Plan and manage costs (marketplace)](https://learn.microsoft.com/azure/ai-studio/how-to/costs-plan-manage#monitor-costs-for-models-offered-through-the-azure-marketplace)